In [110]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from dataclasses import dataclass
import math
import copy
from tqdm.notebook import tqdm
import numpy as np

In [111]:
# A way to represent a state of the game
Nimply = namedtuple("Nimply", "row, num_objects")

In [112]:
# A way to represent a game itself
class Nim:
    # The constructor: it builds the rows and defines the upper bound for the number of objects that can be removed
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    # ToString method
    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    # Returns the rows as a tuple
    def rows(self) -> tuple:
        return tuple(self._rows)

    # Checks if the move is valid and applies it
    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [113]:
NIM_SIZE = 5
MAX_OBJECTS = NIM_SIZE * 2 + 1

In [114]:
# Generates a "score" for the state of the game based on the transformation of the rows into binary strings and the xor of the rows
def nim_sum(state: Nim) -> int:
    # Transforms the rows into binary strings of length 32
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    # Xor of the rows
    xor = tmp.sum(axis=0) % 2
    # Return the value of the xor as an integer (0 means that the game is lost)
    return int("".join(str(_) for _ in xor), base=2)

# Generates a dictionary with the nim_sum (score) associated to all possible moves
def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    # For each possible move
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked

# Selects a move from the possible moves
def optimal(state: Nim) -> Nimply:
    # Generates the possible moves and their score
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    # Filters for moves that don't make the game lost (score = 0)
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    # Selects a random move from the possible moves
    ply = random.choice(spicy_moves)
    return ply

In [115]:
# in version 3 removed randomness
@dataclass
class Strategy:
    # randomness: float
    rows_scores: list[float]
    objects_to_take: float
    aggressive: bool

In [116]:
# Definition of parameters
NUMBER_OF_FITNESS_GAMES = 20
INITIAL_POPULATION = 6 * 2 # always divisible by 2
NUMBER_OF_GENERATIONS = 20
best_strategy = None

In [117]:
# Generates the initial population of strategies
def generate_initial_population(number_of_individuals: int) -> list[Strategy]:
    population = []
    for _ in range(number_of_individuals):
        population.append(
            Strategy([random.random() for _ in range(NIM_SIZE)], 
                     random.random(), 
                     random.choice([True, False])))
    return population

In [118]:
# generates a list with all the possible moves (of type Nimply) at the current state of the game
def possible_moves(state: Nim) -> list:
    possible_moves = []
    for ply in (Nimply(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)):
        try:
            tmp = deepcopy(state)
            tmp.nimming(ply)
            possible_moves.append(ply)
            # print("assertion ok")
        except AssertionError:
            print("assertion error")
    return possible_moves

In [119]:
# this should let strategies play against each other and assign a score to each of them based on the number of wins
# version 3: give a penalty for who loses
def fitness_function_3(strategy: Strategy, num_games = NUMBER_OF_FITNESS_GAMES) -> int:
    score = 0
    for _ in range(num_games):
        # everytime the strategy wins it gets a point
        if nim_match_2(optimal, strategy, fitness = True) == 1:
            score += 1
        else:
            score -= 1
    return score

In [120]:
def crossover(mother: Strategy, father: Strategy):
    child1 = Strategy(rows_scores = father.rows_scores[0:2] + mother.rows_scores[2:],
                      objects_to_take = father.objects_to_take,
                      aggressive = random.choice([mother.aggressive, father.aggressive]))
    child2 = Strategy(rows_scores = mother.rows_scores[0:2] + father.rows_scores[2:],
                        objects_to_take = mother.objects_to_take,
                        aggressive = random.choice([mother.aggressive, father.aggressive]))
    return child1, child2

In [121]:
def calculate_scores(population: list[Strategy]) -> list:
    scores = []
    for strategy in population:
        score = fitness_function_3(strategy)
        scores.append((strategy, score))
    
    # sort scores by fitnes descending
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores

In [122]:
def mutate(strategy: Strategy) -> Strategy:
    mutated_strategy = deepcopy(strategy)
    index_of_mutation = random.choice([i for i in range(len(mutated_strategy.rows_scores))])
    actual_value = mutated_strategy.rows_scores[index_of_mutation]
    if random.random() < 0.5:
        mutated_strategy.rows_scores[index_of_mutation] -= actual_value * 0.1
    else:
        mutated_strategy.rows_scores[index_of_mutation] += actual_value * 0.1
    return mutated_strategy
    

In [123]:
# this returns the best strategy in a population after the application of the genetic algorithm
# version 3.0: cleaned code, more function calls, removed randomness
def evolve_3() -> Strategy:
    # generate random strategies (initial population)
    population = generate_initial_population(INITIAL_POPULATION)
    # print("initial population:", population)

    # calculate the fitness of each strategy
    scores = []
    scores = calculate_scores(population)

    # start the genetic algorithm
    for _ in tqdm(range(NUMBER_OF_GENERATIONS)):
        new_generation = []

        # select the best strategies
        # we take the first half of the population (the best half)
        best_half = scores[:len(scores)//2]

        while len(best_half) > 0:
            # crossover
            # we pick randomly the mother and the father from the best half
            mother = random.choice(best_half)
            best_half.pop(best_half.index(mother))
            father = random.choice(best_half)
            best_half.pop(best_half.index(father))

            # generate two children
            child1, child2 = crossover(mother[0], father[0])

            # mutation
            # parameters of the mutation to try:
            # number of parameters: 1, 2, 3, 4
            # type of mutation: random, gaussian
            # mutation probability: 0.1, 0.2, 0.3, 0.4, 0.5
            # mutation amplitude: 0.1, 0.2, 0.3, 0.4, 0.5

            child1 = mutate(child1)
            child2 = mutate(child2)

            new_generation.append(child1)
            new_generation.append(child2)
            new_generation.append(mother[0])
            new_generation.append(father[0])

        # calculate the fitness of each strategy
        scores = []
        scores = calculate_scores(new_generation)

    # return the best strategy
    # print("scores after training: ", scores)
    # print("best strategy after training: ", scores[0][0])
    return scores[0][0]

In [124]:
def choose_based_on_scores(rows_scores: list[float]) -> int:
    # calculates a weighted probability for each row based on the scores
    total = sum(rows_scores)
    probabilities = [score / total for score in rows_scores]
    indices = list(range(len(rows_scores)))
    chosen_index = random.choices(indices, probabilities)[0]
    return chosen_index

In [125]:
# I think the bottleneck is here, the evolution strategy does not works as expected
def evolution_strategy_3(state: Nim, strategy: Strategy) -> Nimply:
    # generate the possible moves
    allowed_moves = possible_moves(state)
    # print(allowed_moves)
    
    # pick the best move based on the other parameters
    # select the rows that are involved in the possible moves
    not_allowed_rows = set(range(NIM_SIZE)) - set(move.row for move in allowed_moves)
    
    # select the scores of the rows that are involved in the possible moves
    selected_rows_scores = deepcopy(strategy.rows_scores)
    for i in not_allowed_rows:
        selected_rows_scores[i] = 0
    
    # choose the best row
    selected_row = choose_based_on_scores(selected_rows_scores)

    # choose the best number of objects to take
    # count the number of objects in the selected row at the actual state
    number_of_objects = len([move for move in allowed_moves if move.row == selected_row])

    if strategy.aggressive:
        # choose by rounding up
        selected_objects = math.ceil(strategy.objects_to_take * number_of_objects)
    else:
        # choose by rounding down
        selected_objects = math.floor(strategy.objects_to_take * number_of_objects)
    # print("selected_objects: ", selected_objects)
    # if the result of picking obkects is 0, pick 1 (minimum)
    if selected_objects == 0:
        selected_objects = 1
    elif selected_objects > MAX_OBJECTS:
        selected_objects = MAX_OBJECTS

    # reconstruct the corresponding move
    selected_move = Nimply(selected_row, selected_objects)
    # print("selected_move: ", selected_move)
    return selected_move

In [126]:
def nim_match_2(strategy1, strategy2, nim_size = NIM_SIZE, player = 0, fitness = False, debug = False, winner = False):
    agents = (strategy1, strategy2)
    if debug:
        print("agent1: ", agents[0])
        print("agent2: ", agents[1])
    nim = Nim(nim_size)
    logging.info(f"init : {nim}")
    if debug:
        print(f"init : {nim}")
    while nim:
        if player == 0:
            selected_move = agents[player](nim)
        else:
            selected_move = evolution_strategy_3(nim, agents[player])
        logging.info(f"ply: player {player} plays {selected_move}")
        if debug:
            print(f"ply: player {player} plays {selected_move}")
        nim.nimming(selected_move)
        logging.info(f"status: {nim}")
        if debug:
            print(f"status: {nim}")
        player = 1 - player
    logging.info(f"status: Player {player} won!")
    if debug or winner:
        print(f"status: Player {player} won!")
    if fitness:
        return player

In [127]:
best_strategy_3 = evolve_3()

print("best_strategy_2:", best_strategy_3)

  0%|          | 0/20 [00:00<?, ?it/s]

best_strategy_2: Strategy(rows_scores=[0.40782481780390156, 0.6348334726254108, 0.2866384803179473, 0.17314594870864267, 0.6710876699542004], objects_to_take=0.8754948122808679, aggressive=False)


In [128]:
matches = 1000
wins = 0

for _ in tqdm(range(matches)):
    wins += nim_match_2(optimal, best_strategy_3, debug = False, fitness = True)

print("matches played: ", matches)
print("---------------------------------")
print("wins: ", wins)
print("percentage of wins_1: ", wins/matches * 100, "%")
print("---------------------------------")

  0%|          | 0/1000 [00:00<?, ?it/s]

matches played:  1000
---------------------------------
wins:  381
percentage of wins_1:  38.1 %
---------------------------------
